# Python 文件与路径操作完整教程

本 Notebook 整合了 5 个核心主题：
- 文件读取（多种方式）
- with 上下文管理器
- 文件写入（模式详解）
- 目录操作（os/shutil）
- 现代路径操作（pathlib）

## 该笔记本非必要勿运行

## 01. 读取文件

In [ ]:
# Python中操作文件的标准流程：
# 1.创建『文件对象』
# 2.操作文件（读取、写入 等）
# 3.关闭文件

# 文件操作的核心 ———— open函数：它可以打开或创建文件，且支持多种操作模式，返回值是『文件对象』。
# open 函数最常用的三个参数如下：
# 1.file：要操作的文件路径
# 2.mode：文件的打开模式
# r ：读取（默认值）
# w ：写入，并先截断文件
# x ：排它性创建，如果文件已存在，则创建失败
# a ：打开文件用于写入，如果文件存在，则在文件末尾追加内容
# b ：二进制模式
# t ：文本模式（默认值）
# + ：打开用于更新（读取与写入）
# 3.encoding：字符编码

# 读取操作1：使用『文件对象』的 read 方法，读取文件中的内容。
# read 方法说明：
# 1.read(size)中的 size 是可选参数。
#  若不传递 size 参数，表示：读取文件中所有的内容（注意内存占用！）。
#  若传递了 size 参数，表示：读取文件中指定个数的字符，或指定大小的字节。
# 2.read 会从上一次 read 的位置继续读取（指针思想），若到达文件末尾后继续读取，将返回空字符串。

# region
# 第一步：创建『文件对象』
file = open(file='test/a.txt', mode='rt', encoding='utf-8')
file = open('test/a.txt', 'rt', encoding='utf-8')
file = open('test/a.txt', 'rt', encoding='utf-8')
file = open('test/g.jpg', 'rb')

# 第二步：操作文件（读取）
# 多次调用read去逐步读取文件
r1 = file.read(2)
r2 = file.read(3)
r3 = file.read(4)
r4 = file.read()
print(r1, end='')
print(r2, end='')
print(r3, end='')
print(r4, end='')

# 用循环配合多次read（对内存友好）
while True:
    result = file.read(10)
    if result == '':
        break
    print(result, end='')

# 第三步：关闭文件
file.close()
# endregion

# 读取操作2：使用文件对象的 readline 方法，读取文件中的一行。
# readline 方法说明：
# 1.readline(size) 中的 size 是可选参数。
#  若不传递 size 参数，表示：读取当前这一行所有的内容。
#  若传递了 size 参数，表示：表示读取当前行时，最多能读取的字符数，或字节数（size不是行数）。
# 2.readline 方法，也是从上一次位置继续读取，若到达文件末尾后继续读取，也是返回空字符串。

# region
# 第一步：创建『文件对象』
file = open('test/a.txt', 'rt', encoding='utf-8')

# 第二步：操作文件（读取）
# 依次调用readline逐行读取
r1 = file.readline()
r2 = file.readline()
r3 = file.readline()
r4 = file.readline()
print(r1.strip())
print(r2.strip())
print(r3.strip())
print(r4.strip())

# 通过循环配合readline逐行读取
while True:
    line = file.readline()
    if line == '':
        break
    # print(line.strip())
    print(line, end='')

# 第三步：关闭文件
file.close()
# endregion

# 读取操作3：使用 for 循环直接遍历文件对象
# region
# 第一步：创建『文件对象』
file = open('test/a.txt', 'rt', encoding='utf-8')

# 第二步：操作文件（读取）
for line in file:
    print(line, end='')

# 第三步：关闭文件
file.close()
# endregion

# 读取操作4：使用文件对象的 readlines 方法，一次性按“行”读完，返回一个列表。
# readlines 方法说明：
# 1.readlines(hint) 中的 hint 是可选参数。
#  若不传递 hint 参数，表示：读取当前文件的所有行。
#  若传递了 hint 参数，表示：期望读取的【字符个数 或 字节数的上限】（hint不是行数）。
# 2.注意：由于 readlines 是一次性读取文件的所有内容，所以不适合读取体积较大的文件。

# region
# 第一步：创建『文件对象』
file = open('test/a.txt', 'rt', encoding='utf-8')

# 第二步：操作文件（读取）
result = file.readlines()
print(result)

# 第三步：关闭文件
file.close()
# endregion

# 最佳实践：使用 with 上下文管理器，结合for循环遍历，逐行读取文件。
with open('test/a.txt', 'rt', encoding='utf-8') as file:
    for line in file:
        print(line, end='')

## 02. 关于 with 上下文管理器

In [ ]:
# 1.Python 中的 with 主要用于管理程序中“需要成对出现的操作”，例如：
#  上锁 / 解锁
#  打开 / 关闭
#  借用 / 归还
# 2.最终目标：编码者只管做具体的事，“进入”和“离开”的事，让 Python 自动处理。
# 3.语法格式如下：
# with 能得到一个上下文管理器的表达式 as 变量:
#     具体的事1
#     具体的事2
#     具体的事3
# 4.上下文管理器协议：
# (1). __enter__ 方法：with 中的代码执行【之前】调用，其返回值会赋值给 as 后的变量。
# (2). __exit__ 方法：with 中的代码执行【结束后】调用（无论是 with 中否出现异常都会调用）。

# 定义一个 Person 类，让其实例对象遵循：上下文管理器协议
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def speak(self):
        print(f'我叫{self.name}，年龄是{self.age}')

    def __enter__(self):
        print('-----我是进入的逻辑-----')
        return self

    # 当 with 中的代码发生异常时，__exit__ 方法的返回值规则如下：
    #  返回“真”：表示异常【已经】被处理，异常【不会】被继续抛出。
    #  返回“假”：表示异常【没有】被处理，异常【会】被继续抛出。
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('-----我是离开的逻辑-----')
        # exc_type : 异常类型
        # exc_val : 异常对象
        # exc_tb : 异常追踪信息
        if exc_type:
            print(f'异常类型：{exc_type}')
            print(f'异常对象：{exc_val}')
            print(f'异常追踪信息：{exc_tb}')
            return True  # 表示异常已处理

# 执行流程：
# 1.计算 with 后面的表达式，得到一个『上下文管理器』。
# 2.调用『上下文管理器』的 __enter__() 方法，并将其返回值赋给 as 后面的变量。
# 3.执行 with 所管理的代码。
# 4.无论代 with 中的代码，是正常结束，还是发生异常，都会自动调用『上下文管理器』的 __exit__ 方法。
with Person('张三', 18) as p1:
    p1.speak()
    # p1.study()  # 故意触发 AttributeError
    print(666)

## 03. 写入文件

In [ ]:
import time

# open 函数最常用的三个参数如下：
# 1.file：要操作的文件路径
# 2.mode：文件的打开模式
# r ：读取（默认值）
# w ：写入，并先截断文件
# x ：排它性创建，如果文件已存在，则创建失败
# a ：打开文件用于写入，如果文件存在，则在文件末尾追加内容
# b ：二进制模式
# t ：文本模式（默认值）
# + ：打开用于更新（读取与写入）
# 3.encoding：字符编码

# 测试 w 模式
with open('test/a.txt', 'wt', encoding='utf-8') as file:
    file.write('你好')

# 测试 x 模式
with open('test/demo.txt', 'xt', encoding='utf-8') as file:
    file.write('你好')

# 测试 a 模式
with open('a.txt', 'at', encoding='utf-8') as file:
    file.write('你好')

# 在 Python 中文件写入时，并不是每写一次就立刻落盘，而是：先写到“缓冲区”里。
# 文件对象的 flush 方法：把缓冲区中的数据，立刻写入到文件中。
with open('test/demo.txt', 'at', encoding='utf-8') as file:
    file.write('你好1')
    file.write('你好2')
    file.flush()
    time.sleep(10000)
    file.write('你好3')
    file.write('你好4')

# 测试 rt+
with open('test/a.txt', 'rt+', encoding='utf-8') as file:
    # seek(offset, whence)方法：用于改变文件对象指针的位置，参数说明如下：
    # offset：偏移量，要移动多少距离
    # whence：参考点，从哪里开始计算偏移，有三种取值：
    # 0：从文件开头计算（默认值）
    # 1：从当前位置计算
    # 2：从文件末尾计算
    # 注意：在文本模式下，不要随意去定位中文字符位置，否则可能破坏文件编码。
    file.seek(0, 0)
    file.write('你好')

# 测试 wt+
# with open('a.txt', 'wt+', encoding='utf-8') as file:
#     file.write('你好')
#     file.seek(0, 0)
#     result = file.read()
#     print(result)

# 测试 xt+
# with open('demo3.txt', 'xt+', encoding='utf-8') as file:
#     file.write('你好')
#     file.seek(0, 0)
#     result = file.read()
#     print(result)

# 测试 at+
with open('test/a.txt', 'at+', encoding='utf-8') as file:
    file.write('你好')
    file.seek(0, 0)
    result = file.read()
    print(result)

## 04. 目录操作（os / shutil）

In [ ]:
import os
import shutil

#  os.mkdir(path)：创建“单级”目录（如果目录已经存在，则会抛出异常）
try:
    os.mkdir('test/demo')
except FileExistsError:
    print('目录已存在')

#  os.makedirs(path)：创建“多级”目录（如果路径中的所有目录都已经存在，则会抛出异常）
try:
    os.makedirs('test/demo/aa/bb')
except FileExistsError:
    print('目录已存在')

#  os.rmdir(path)：删除空目录（如果目录不存在，或目录非空，都会抛出异常）
try:
    os.rmdir('test/demo/aa/bb')
except FileNotFoundError:
    print('目录不存在')
except OSError:
    print('目录非空')

#  os.removedirs(path)：递归删除空目录，在成功删除末尾一级目录后，会“向上”尝试把父级目录也删除（直到父目录不是空目录）
os.removedirs('test/demo/aa/bb')

#  os.path.exists(path)：判断路径是否存在（文件/目录都算）
result = os.path.exists('test/demo/aa/bb')
print(result)

#  os.path.isdir(path)：用于判断路径，具体规则如下：
# 1.路径不存在 ==================> 返回 False
# 2.路径存在，但指向的是文件 =====> 返回 False
# 3.路径存在，并且是目录 =======> 返回 True
result = os.path.isdir('test/demo/aa/bb')
print(result)

#  os.path.isfile(path)：判断是否为文件
result = os.path.isfile('test/demo/aa/bb')
print(result)

#  os.scandir(path)：扫描指定目录
result = os.scandir('test/demo')
for item in result:
    print('目录' if item.is_dir() else '文件', item.name)

#  os.walk(path)：按层级，递归地遍历指定目录下，所有的子目录和文件
result = os.walk('D:/demo')
for item in result:
    print(item)

#  危险操作：删除有内容的目录
shutil.rmtree('D:/demo')

# 示例：安全创建目录
demo_dir = 'test/demo'
if not os.path.exists(demo_dir):
    os.makedirs(demo_dir)
    print(f'目录 {demo_dir} 创建成功')
else:
    print(f'目录 {demo_dir} 已存在')

# 示例：列出目录内容
with os.scandir('.') as entries:
    for entry in entries:
        print(f"{'目录' if entry.is_dir() else '文件'}: {entry.name}")

## 05. 现代路径操作：pathlib

In [ ]:
from pathlib import Path
import shutil

# pathlib 是 Python 3.4+ 引入的面向对象的路径操作库，比 os.path 更直观、更强大。

# 1. 创建 Path 对象
p = Path(r'.\test')                    # 当前目录
home = Path.home()               # 用户主目录
file_path = Path('data') / 'input.txt'  # 路径拼接（跨平台！）

print("当前路径:", p)
print("主目录:", home)
print("拼接路径:", file_path)

# 2. 路径属性
full_path = Path('test/main.py')
print("\n路径属性:")
print("名称:", full_path.name)        # main.py
print("后缀:", full_path.suffix)      # .py
print("无后缀名:", full_path.stem)    # main
print("父目录:", full_path.parent)    # /home/user/project
print("绝对路径:", full_path.resolve())

# 3. 路径存在性与类型判断
print("\n存在性检查:")
print(f"{p} 存在吗? {p.exists()}")
print(f"{p} 是目录吗? {p.is_dir()}")
print(f"{p} 是文件吗? {p.is_file()}")

# 4. 目录操作
demo = Path('pathlib_demo')

# 创建目录（parents=True 可创建多级，exist_ok=True 避免已存在报错）
demo.mkdir(parents=True, exist_ok=True)
print(f"\n创建目录: {demo}")

# 创建子目录
(demo / 'subdir').mkdir(exist_ok=True)

# 5. 文件操作
txt_file = demo / 'hello.txt'

# 写入文件（自动创建父目录，无需手动 open/close）
txt_file.write_text('Hello, pathlib!\n这是第二行。', encoding='utf-8')
print(f"写入文件: {txt_file}")

# 读取文件
content = txt_file.read_text(encoding='utf-8')
print("文件内容:\n", content)

# 6. 遍历目录
print("\n遍历目录:")
for item in demo.iterdir():
    print(f"{'[DIR] ' if item.is_dir() else '[FILE]'} {item.name}")

# 递归查找所有 .txt 文件
print("\n查找所有 .txt 文件:")
for txt in demo.rglob('*.txt'):
    print(txt)

# 7. 删除操作
# 删除文件
if txt_file.exists():
    txt_file.unlink()
    print(f"删除文件: {txt_file}")

# 删除空目录
subdir = demo / 'subdir'
if subdir.exists():
    subdir.rmdir()
    print(f"删除空目录: {subdir}")

# 删除非空目录（需用 shutil）
if demo.exists():
    shutil.rmtree(demo)
    print(f"删除非空目录: {demo}")

# 8. 与传统 open 结合（推荐方式）
config = Path('config.json')
if config.exists():
    with config.open('r', encoding='utf-8') as f:
        data = f.read()
        # ... 处理数据

print("\n✅ pathlib 是现代 Python 路径操作的首选！")